In [ ]:
!pip install tokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.7 MB/s eta 0:00:00


In [ ]:
import string

import numpy as np
import pandas as pd
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import ConfusionMatrixDisplay, precision_score, recall_score, f1_score

from tokenizer import tokenizer as tk

import string
import numpy as np
import pandas as pd
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import ConfusionMatrixDisplay, precision_score, recall_score, f1_score

# Función de tokenización personalizada

#Lectura de los Datos.

In [ ]:
df = pd.read_csv('./MovieReviews_clean.csv')
print(df.shape)


(5000, 3)


In [ ]:
df.head()

,Unnamed: 0,review_es,sentimiento
0,0,si está buscando una película de guerra típica...,positivo
1,1,supongo que algunos directores de películas de...,positivo
2,2,es difícil contarle más sobre esta película si...,positivo
3,3,la película comienza muy lentamente con el est...,positivo
4,4,esta película es verdadera acción en su máxima...,positivo


In [ ]:
df = df.rename({'Unnamed: 0': 'id'},axis=1)

df.head()

,id,review_es,sentimiento
0,0,si está buscando una película de guerra típica...,positivo
1,1,supongo que algunos directores de películas de...,positivo
2,2,es difícil contarle más sobre esta película si...,positivo
3,3,la película comienza muy lentamente con el est...,positivo
4,4,esta película es verdadera acción en su máxima...,positivo


In [ ]:
df['sentimiento'].value_counts(dropna = False, normalize = True)

positivo    0.5
negativo    0.5
Name: sentimiento, dtype: float64

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
# Descargando las stopwords
nltk.download('stopwords')
sp_stop_words = set(stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
spanish_stopwords = stopwords.words('spanish')

In [ ]:
# Función para tokenizar los comments
def tokenizer(text):
     # Replace all non-alphanumeric characters with whitespace
    text = re.sub(r'[^\w\s]', ' ', text)

    # Convert all text to lowercase
    text = text.lower()

    # Split text into tokens
    tokens = text.split()

    return tokens

In [ ]:
comments_labeled_df = df.loc[df['sentimiento'].notnull()]

In [ ]:
# comments a ser usados para entrenar y evaluar el modelo
comments_labeled_df.shape

(5000, 3)

In [ ]:
#Working only with positivo and negativo
comments_labeled_df = comments_labeled_df.loc[comments_labeled_df['sentimiento'].isin(['positivo', 'negativo'])]

In [ ]:
comments_labeled_df['sentimiento'].value_counts(dropna = False, normalize = True)

positivo    0.5
negativo    0.5
Name: sentimiento, dtype: float64

In [ ]:
comments_labeled_df['sentimiento'].value_counts(dropna = False, normalize = True)

positivo    0.5
negativo    0.5
Name: sentimiento, dtype: float64

In [ ]:
comments_labeled_df.head()


,id,review_es,sentimiento
0,0,si está buscando una película de guerra típica esta no es así por lo que una nota a todos los aficionados a la guerra de ansiancia de carnajería de testosterona no se molesten aunque la película se trata de personajes rusos en la segunda guerra mundial no espere ver a ningún nazis cañones sangre sangre etc no es una película sobre las personas que causan una guerra o que luchan contra una guerra es una película sobre las personas ordinarias que le ocurren la guerra y las elecciones que hacen para tratar con ella cinematografía escritura todos los s perfectos aquí sin duda lo apreciarás si eres ruso como yo pero incluso si no probablemente lo ames si no habla ruso busque la versión dvd del dvd de ruscico russian cinema cinema tiene subtítulos en unos idiomas diferentes pero el doblaje inglés en este que diría es igual de bueno por supuesto no es tan bueno como la pista rusa original algunas cosas se pierden en la traducción sino tan buenas como los subtítulos en inglés así que vaya a revisarlo especialmente si está estudiando una película en cualquier aspecto,positivo
1,1,supongo que algunos directores de películas de lujo estaban sentados buscando su abrigo grappa aramangac o jugo de selva alguna noche en los años durante los cannes u otro festival de cine y uno dijo oye chicos hagamos una película donde cada uno de nosotros cree un segmento alrededor una clase mundial aria welllll tipo de tipo de trabajo claramente alguien fue lo suficientemente inteligente como para seleccionar algunas de las mejores grabaciones de las arias elegidas por ejemplo nessun dorma de bjoreling así que si estuvieras ciego y mintiendo en el piso simplemente escuchando el dvd obtuviste más que el valor de tu dinero no todos los directores lo sucedieron pero más lo hicieron lo que no y la película parece mejorar con cada visión a lo largo de los años mi favorito es el dúo de amor inquietantemente hermoso de die tod statd está bien un joven desnudo elizabeth hurley es ojos pero su marido cantó a ella el fantasma de su esposa es increíblemente hermoso con la música del amor segundo solo para gia nella notte densa de otello y desdemona en todo el repertorio operístico podría haber sido mejor el flick seguro lo que no podría haber sido pero vale la pena una visión especialmente de un estado de ánimo hiper romántico,positivo
2,2,es difícil contarle más sobre esta película sin estropearla lo disfruté porque no esperaba lo que estaba viendo sino un drama sexual ordinario así que es un thriller de pscyho sexual en el que nada es lo que parece cuenta con emmanuelle seigner sin extraño al género y a la desnudez en la que su esposo polanski la había dirigido y un rendimiento espeluznante dije espeluznante sí espeluznante de toreton el actor de bernard tavernier parece que un pascal bruckner se encuentra con roman polanski mejor que la luna amarga como un chabrol que se fue por extravisos o thriller de clouzot he visto a alguien que menciona les diaboliques pero más cerca de georges franju s les yeux sans visage ojos sin cara lapadrino del dr phibes y más una gema solo me temo que lo hicieran en un remake de hollywood como lo hicieron con nighwatch y la desaparición,positivo
3,3,la película comienza muy lentamente con el estilo de vida de wallace napalm un asistente en una estación de entrega de fotografía su esposa ha sido restringida a su casa con un brazalete de tobillo como resultado de una oración para el incendio wallace es miembro del departamento de bomberos voluntarios y toma en serio incendio en serio observamos el procedimiento de vida bastante aburrida de wallace de repente llega algo nuevo y jarring un carnaval viajero llega a la ciudad una de sus estrellas es wilder napalm el hermano de wallace es un payaso pero tiene un talento especial sobre wallace ambos son pirocinadores ni pirótica personas capaces de iniciar incendios a través de la energía mental wallace mantiene su secreto de poderes wilder le permite conocer a 

### Diviendo los comments etiquetados en los conjuntos de entrenamiento y prueba

In [ ]:
# Cuando se cuenta con muy pocos datos etiquetados, por lo general se debe disponer de una menor cantidad de datos para evaluación
X_train, X_test, y_train, y_test = train_test_split(comments_labeled_df['review_es'], comments_labeled_df['sentimiento'], test_size = 0.2, random_state = 1)

In [62]:

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(analyzer='word', tokenizer=tokenizer, stop_words=spanish_stopwords, lowercase=True)),
    ('clf', RandomForestClassifier(random_state=3, n_estimators=500))
])

# Ajustar el pipeline utilizando GridSearchCV para encontrar los mejores parámetros
parameters = {
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'clf__max_depth': [None, 10, 20],
    'clf__min_samples_split': [2, 5, 10]
}
grid_search = GridSearchCV(pipeline, parameters, cv=5)
grid_search.fit(X_train, y_train)

# Obtener las predicciones
y_train_pred = grid_search.predict(X_train)
y_test_pred = grid_search.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extr

In [63]:

import joblib

# Guardar el pipeline en un archivo
joblib.dump(grid_search, 'pipeline_model.joblib')

['pipeline_model.joblib']

In [65]:
import pickle

with open('pipeline_model.joblib', 'wb') as file:
    pickle.dump(grid_search, file)

---------------------------------------------------
